In [2]:
import sys
sys.path.append("..")
%cd ..

c:\Users\frede\OneDrive - Danmarks Tekniske Universitet\DTU\6. Semester\Bachelor2021-1


In [3]:
from DataPrep.load_data import load_data
import pyforest

In [44]:
df = pd.read_csv("data\createdDat\Data_Oskar\df_11052021.csv")
plug = pd.read_csv("data\createdDat\Data_Oskar\plugdf_11052021.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
def get_censored(df):
    df = df.drop(columns=["Unnamed: 0","Latitude","Longitude","CenterLat","CenterLon", "Port Number", "Plug Type"])
    df["Start Date"] = pd.to_datetime(df["Start Date"]) 
    test = pd.DataFrame(columns=list(df.columns))
    for label in range(8):
        maks = plug["Date"][plug["Label"]==label][plug[plug["Label"]==label]["Plugs"]==plug[plug["Label"]==label]["Plugs"].max()]
        intercept = maks[maks.index[label]]
        macs_until_intercept = df[(df["Label"]==label) & (df["Start Date"].dt.date<pd.to_datetime(intercept))]["MAC Address"].unique()
        liste = df[df["Label"]==label]["MAC Address"].isin(macs_until_intercept)

        test = test.append(df[(df["Label"]==label) & (liste)])
    
    return test
    



In [58]:
test = get_censored(df)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
test

,Station Name,MAC Address,Start Date,End Date,Total Duration (mins),Charging Time (mins),Energy (kWh),Fee (USD),Label
390,PALO ALTO CA / BRYANT #2,000D6F0000A2108E,2011-10-10 15:23:00,2011-10-10 15:23:00,274.633333,157.100000,9.155255,0.00,0
392,PALO ALTO CA / BRYANT #2,000D6F0000A2108E,2011-10-10 19:14:00,2011-10-10 21:18:00,124.216667,123.966667,7.609224,0.00,0
400,PALO ALTO CA / BRYANT #2,000D6F0000A2108E,2011-10-11 15:48:00,2011-10-11 19:32:00,224.383333,218.883333,13.397377,0.00,0
404,PALO ALTO CA / BRYANT #2,000D6F0000A2108E,2011-10-12 07:44:00,2011-10-12 10:54:00,189.516667,121.216667,7.312490,0.00,0
409,PALO ALTO CA / BRYANT #2,000D6F0000A2108E,2011-10-12 15:26:00,2011-10-12 17:49:00,143.866667,143.866667,8.703846,0.00,0
...,...,...,...,...,...,...,...,...,...
132777,PALO ALTO CA / WEBSTER #1,0024B10000021146,2017-08-03 17:40:00,2017-08-03 20:47:00,186.066667,149.266667,7.714000,1.80,7
132779,PALO ALTO CA / WEBSTER #3,0024B100000213D1,2017-08-03 17:44:00,2017-08-03 20:47:00,182.166667,128.533333,12.505000,3.43,7
132795,PALO ALTO CA / WEBSTER #3,0024B100000213D1,2017-08-03 19:04:00,2017-08-03 19:38:00,33.933333,33.750000,2.085000,0.48,7
132800,PALO ALTO CA / WEBSTER #3,0024B100000213D1,2017-08-03 19:46:00,2017-08-03 21:17:00,91.483333,80.583333,3.999000,0.92,7


In [95]:
def downsampling_to_days(df):
    labels = df["Label"].unique()
    dfClean2 = pd.DataFrame(columns = df.columns[1:])
    for label in labels:
        d = df[df["Label"] == label].resample("D", on="Start Date").agg({'Charging Time (mins)':'sum','Total Duration (mins)':'sum', 'Energy (kWh)':'sum', 'Fee (USD)':'sum', 'MAC Address':'min', 'Station Name':'min'}).fillna('')
        d["Label"] = label
        
        dfClean2 = dfClean2.append(d)
    
    dfClean2 = dfClean2.reset_index()
    dfClean2 = dfClean2.rename(columns = {"index":"Start Date"})
    
    new_col_order = ['Start Date','Label','Charging Time (mins)','Total Duration (mins)', 'Energy (kWh)', 'Fee (USD)', 'MAC Address', 'Station Name']
    dfClean3 = dfClean2[new_col_order]
    return dfClean3

In [105]:
df_down = downsampling_to_days(test)
df_down = df_down.dropna(axis=1)
df_down=df_down.set_index("Start Date")

<IPython.core.display.Javascript object>

In [106]:
df_down

,Label,Charging Time (mins),Total Duration (mins),Energy (kWh),Fee (USD),MAC Address,Station Name
Start Date,,,,,,,
2011-10-10,0,281.066667,398.850000,16.764479,0.00,000D6F0000A2108E,PALO ALTO CA / BRYANT #2
2011-10-11,0,218.883333,224.383333,13.397377,0.00,000D6F0000A2108E,PALO ALTO CA / BRYANT #2
2011-10-12,0,265.083333,333.383333,16.016336,0.00,000D6F0000A2108E,PALO ALTO CA / BRYANT #2
2011-10-13,0,430.066667,552.133333,26.342279,0.00,000D6F0000A20D9E,PALO ALTO CA / BRYANT #1
2011-10-14,0,195.550000,195.716667,11.908109,0.00,000D6F0000A20D9E,PALO ALTO CA / BRYANT #1
...,...,...,...,...,...,...,...
2017-07-30,7,1710.466667,2075.066667,128.477000,0.00,0024B10000021146,PALO ALTO CA / WEBSTER #1
2017-07-31,7,2708.300000,3707.433333,189.945000,0.00,0024B10000021146,PALO ALTO CA / WEBSTER #1
2017-08-01,7,2318.816667,2385.400000,147.557000,33.94,0024B10000021146,PALO ALTO CA / WEBSTER #1


In [116]:
df_down.to_csv("data\createdDat\Censorship_scheem\Plugs_censord.csv")